In [1]:
class Example:
    name = "XXX"

    @staticmethod
    def print_static(): # можно не указывать self, аналог обычной функции внутри класса
        print("static")

    @classmethod
    def print_cls(cls): # возвращает или работает с атрибутами класса а не его экземпляра
        print(f"class_method for {cls.__name__}")

    def __init__(self, val):
        self.val = val
        self._protected_val = val + 100
        self.__private_val = val + 1000
        
    def print_offset(self, offset=10):
        print(self.val + offset, self.__private_val)

    def __str__(self):
        return f"{self.__class__.__name__}:val={self.val}"
    
    
exp = Example(9)

In [2]:
exp.print_static()
Example.print_static()

static
static


In [3]:
Example.print_cls()
exp.print_cls()

class_method for Example
class_method for Example


In [4]:
exp.print_offset()

19 1009


In [5]:
Example.print_offset, exp.print_offset

(<function __main__.Example.print_offset(self, offset=10)>,
 <bound method Example.print_offset of <__main__.Example object at 0x0000022388FC9430>>)

In [6]:
Example.print_offset(exp)

19 1009


In [7]:
exp.val, exp._protected_val

(9, 109)

In [8]:
exp.__private_val

AttributeError: 'Example' object has no attribute '__private_val'

In [9]:

exp.__dict__

{'val': 9, '_protected_val': 109, '_Example__private_val': 1009}

In [10]:
exp._Example__private_val = 999999

In [11]:
exp.print_offset()

19 999999


In [12]:
exp.name, Example.name, exp.name is Example.name

('XXX', 'XXX', True)

In [13]:
exp.__dict__

{'val': 9, '_protected_val': 109, '_Example__private_val': 999999}

In [14]:
exp._Example__private_val

999999

In [15]:
Example.__dict__

mappingproxy({'__module__': '__main__',
              'name': 'XXX',
              'print_static': <staticmethod at 0x22388f8dd90>,
              'print_cls': <classmethod at 0x22388f887c0>,
              '__init__': <function __main__.Example.__init__(self, val)>,
              'print_offset': <function __main__.Example.print_offset(self, offset=10)>,
              '__str__': <function __main__.Example.__str__(self)>,
              '__dict__': <attribute '__dict__' of 'Example' objects>,
              '__weakref__': <attribute '__weakref__' of 'Example' objects>,
              '__doc__': None})

In [16]:
exp.name = "QWERTY"

In [17]:
exp.name, Example.name, exp.name is Example.name

('QWERTY', 'XXX', False)

In [18]:
exp.__dict__

{'val': 9,
 '_protected_val': 109,
 '_Example__private_val': 999999,
 'name': 'QWERTY'}

In [19]:
exp.name = "3333"

In [20]:
del exp.name

In [21]:
exp.name, Example.name

('XXX', 'XXX')

In [22]:
exp.__dict__

{'val': 9, '_protected_val': 109, '_Example__private_val': 999999}

In [23]:
## Сколько бы классов Singleton мы не создавали - это будет ссылка на один объект

class Singleton:
    _instance = None
    
    # Переопределяем класс Singleton 
    def __new__(cls, *args, **kwargs):
        print("__new__", cls, args, kwargs)
        if cls._instance is None:
            print("no_instance")
            cls._instance = super().__new__(cls)
        return cls._instance
    
# inst1 = Singleton()
# inst2 = Singleton()
# inst1 is inst2

In [24]:
class SingleConnection(Singleton):
    
    def __init__(self, domain):
        print("__init__")
        self.domain = domain
    
    def __del__(self):
        print("DELETE")
        
inst11 = SingleConnection("pg")
inst22 = SingleConnection("mysql")
inst11 is inst22

__new__ <class '__main__.SingleConnection'> ('pg',) {}
no_instance
__init__
__new__ <class '__main__.SingleConnection'> ('mysql',) {}
__init__


True

In [25]:
inst1 = SingleConnection("pg")
print("domain1", inst1.domain, id(inst1))
del inst1

print(SingleConnection._instance, Singleton._instance)
SingleConnection._instance = None
print("------")



inst2 = SingleConnection("mysql")

b = object()
for i in range(1000):
    d = [1, 2, 3]
    d = object()

print("domain2", inst2.domain, id(inst2))
# inst1 is inst2

__new__ <class '__main__.SingleConnection'> ('pg',) {}
__init__
domain1 pg 2351645368080
<__main__.SingleConnection object at 0x0000022388FC9F10> None
------
__new__ <class '__main__.SingleConnection'> ('mysql',) {}
no_instance
__init__
domain2 mysql 2351646165936


In [26]:
print(id([1, 2, 3, 4]))
print(id([1, 2, 3, 4]))
print(id([1, 2, 3, 4]))
print(id([1, 2, 3, 4]))

print(id((1, 2, 3, 4)))
print(id((1, 2, 3, 4)))
print(id((1, 2, 3, 4)))
print(id((1, 2, 3, 4)))


2351646154432
2351646154240
2351646154560
2351646154176
2351646202112
2351646202192
2351646202112
2351646202192


In [27]:
class Attr:
    def __set_name__(self, owner, name):
        print(f"{locals()}=")
        self.name = name
    
    def __init__(self, val=10):
        self.val = val


class A:
    x = Attr()  # Automatically calls: x.__set_name__(A, "x")
    #y = Attr("yy")


# a = A()

{'self': <__main__.Attr object at 0x0000022388FC9970>, 'owner': <class '__main__.A'>, 'name': 'x'}=


In [28]:
a = A
a.x.name

'x'

In [29]:
class Adder:
    def __init__(self, val):
        self._val = val

    def __call__(self, number):
        return self._val + number
    

add5 = Adder(5)
print(add5(7))
print(add5(0))
print(add5(-5))
        

12
5
0


In [30]:
def add(n1, n2):
    return n1 + n2

In [31]:
add.__call__(5, 7)

12

In [32]:
class Attrs:
    def __init__(self, val):
        self.val = val
    
    def __getattribute__(self, name):
        print("__getattribute__", name)
        return super().__getattribute__(name)

    def __getattr__(self, name):
        print("__getattr__", name)
        if name == "name":
            return 42
        return super().__getattribute__(name)

    def __setattr__(self, name, val):
        print("__setattr__", name, val)
        return super().__setattr__(name, val)
    
    def __delattr__(self, name):
        print("__delattr__", name)
        return super().__delattr__(name)
    

    
attr = Attrs(42)

__setattr__ val 42


In [33]:
attr.val 

__getattribute__ val


42

In [34]:
attr.name == "attribs"

__getattribute__ name
__getattr__ name


False

In [35]:
attr.not_exist

__getattribute__ not_exist
__getattr__ not_exist


AttributeError: 'Attrs' object has no attribute 'not_exist'

In [36]:
attr.val

__getattribute__ val


42

In [37]:
attr.name = "attribs"

__setattr__ name attribs


In [38]:
attr.name

__getattribute__ name


'attribs'

In [39]:
del attr.name

__delattr__ name


In [40]:
class Timing:
    def __init__(self, start, end):
        self.start = start
        self.end = end

    @classmethod
    def __init_subclass__(cls, **kwargs):
        print("INIT subclass", cls, kwargs)
        super().__init_subclass__(**kwargs)


class MinuteTiming(Timing):
    pass


class HourTiming(MinuteTiming):
    pass

INIT subclass <class '__main__.MinuteTiming'> {}
INIT subclass <class '__main__.HourTiming'> {}


In [41]:
class Cursor:
    def __init__(self, name):
        print("Cursor.__init__", name)
        self.name = name
    
    def draw(self):
        print("Cursor.draw", self.name)


class ColoredCursor(Cursor):
    def __init__(self, color, *args, **kwargs):
        super().__init__(*args, **kwargs)

        print("ColoredCursor.__init__", color)
        self.color = color
    
    def draw(self):
        print("ColoredCursor.draw", self.color)
        #canvas.set_color(self.color)
        super().draw()

        
class BoldCursor(Cursor):
    def __init__(self, width, *args, **kwargs):
        super().__init__(*args, **kwargs)

        print("BoldCursor.__init__", width)
        self.width = width
    
    def draw(self):
        print("BoldCursor.draw", self.width)
        #canvas.setwidth(self.width)
        super().draw()
        

class BoldColoredCursor(BoldCursor, ColoredCursor):
    pass

BoldColoredCursor.__mro__

(__main__.BoldColoredCursor,
 __main__.BoldCursor,
 __main__.ColoredCursor,
 __main__.Cursor,
 object)

In [42]:
BoldColoredCursor.__bases__, BoldColoredCursor.__base__

((__main__.BoldCursor, __main__.ColoredCursor), __main__.BoldCursor)

In [43]:
bold_col = BoldColoredCursor(18, "yellow", "text")
print('---')
bold_col.draw()

Cursor.__init__ text
ColoredCursor.__init__ yellow
BoldCursor.__init__ 18
---
BoldCursor.draw 18
ColoredCursor.draw yellow
Cursor.draw text


In [44]:
cur = ColoredCursor("green", "line")

Cursor.__init__ line
ColoredCursor.__init__ green


In [98]:
cur.draw()

ColoredCursor.draw green
Cursor.draw line


In [45]:
bold = BoldCursor(18, "text")
print('---')
bold.draw()

Cursor.__init__ text
BoldCursor.__init__ 18
---
BoldCursor.draw 18
Cursor.draw text


In [46]:
class A:
    pass

class B(A):
    pass

class C(A):
    pass

D.__mro__

NameError: name 'D' is not defined

In [47]:
class D(A, C):  # A, C, A
    pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases A, C

In [49]:
def fib():
    a, b = 0, 1

    while True:
#         print('take')
        a, b = b, a + b
        yield a


f_gen = fib()

        
for i in range(10):
    print(next(f_gen))

1
1
2
3
5
8
13
21
34
55


In [50]:
from itertools import islice

In [51]:
list(islice(fib(), 10))

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [52]:
f = islice(fib(), 10)


In [53]:
next(f)

1

In [54]:
list(islice(fib(), 10))

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [55]:
f = islice(fib(), 10)

list(
    zip(f, map(lambda x: x + 100, f))
)

[(1, 101), (2, 103), (5, 108), (13, 121), (34, 155)]

In [56]:
f = islice(fib(), 10)

list(
    zip(f, f, f, f)
)

[(1, 1, 2, 3), (5, 8, 13, 21)]

In [57]:
f1 = islice(fib(), 10)
f2 = islice(fib(), 10)

list(
    zip(f1, f1, f2, f2)
)

# (1, 1, 1, 1), (2, 3, 2, 3)

[(1, 1, 1, 1), (2, 3, 2, 3), (5, 8, 5, 8), (13, 21, 13, 21), (34, 55, 34, 55)]

In [59]:
lst = iter([1, 2, 3, 4])

In [60]:
next(lst)
next(lst)
next(lst)
next(lst)

4

In [61]:
next(lst)

StopIteration: 

In [63]:
lst = [1, 2, 3, 4]
next(lst)

TypeError: 'list' object is not an iterator

In [ ]:
# for i in range():
#     for j in [1, 2, 3]:

In [64]:
def fib():
    a, b = 0, 1

    while True:
        a, b = b, a + b
        yield a

In [65]:
class CustomIter:
    def __init__(self, lst):
        self._lst = lst
    
    def __iter__(self):
        print('__itert__')
        return iter(self._lst)
    

for i in CustomIter([5, 9, 11, 0, 2]):
    print(i)

__itert__
5
9
11
0
2


In [66]:
class CustomIter:
    def __init__(self, lst):
        self._lst = lst
        self._i = 0

    def __iter__(self):
        print('__itert__')
        return self

    def __next__(self):
        if self._i < len(self._lst):
            val = self._lst[self._i]
            self._i += 1
            return val ** 2
        raise StopIteration()


it = CustomIter([5, 9, 11, 0, 2])
print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))

25
81
121
0
4


StopIteration: 

In [67]:
for i in CustomIter([5, 9, 11, 0, 2]):
    print(i)

__itert__
25
81
121
0
4


In [68]:
iter(CustomIter([5, 9, 11, 0, 2]))

__itert__


In [161]:
#type(x) == int or type(x) == float

In [69]:
isinstance(1, (int, float)), isinstance("1", (int, float))

(True, False)

In [164]:
type(CustomIter([])), type(1)

(__main__.CustomIter, int)

In [165]:
cur = BoldColoredCursor("18", "green", "line")

Cursor.__init__ line
ColoredCursor.__init__ green
BoldCursor.__init__ 18


In [166]:
type(cur)

__main__.BoldColoredCursor

In [169]:
isinstance(cur, BoldColoredCursor), isinstance(cur, Cursor), isinstance(cur, object)

(True, True, True)

In [170]:
def fn(): pass

isinstance(fn, object)

True

In [173]:
isinstance(cur, (str, int, Cursor))

True

In [176]:
bold_cur = BoldCursor(18, "name")
cur = Cursor("name")

Cursor.__init__ name
BoldCursor.__init__ 18
Cursor.__init__ name


In [179]:
print(
    isinstance(bold_cur, Cursor),
    isinstance(bold_cur, BoldColoredCursor),
    isinstance(bold_cur, ColoredCursor),
    isinstance(cur, BoldColoredCursor),
)

True False False False


In [184]:
import json

with open("data.json", "w") as f:
    print(f)
    json.dump({"x": "y"}, f)

<_io.TextIOWrapper name='data.json' mode='w' encoding='UTF-8'>


In [183]:
!cat data.json

{"x": "y"}

In [185]:
import io

In [191]:
buf = io.StringIO()

In [192]:
json.dump({"x": "y"}, buf)
json.dump({"z": "22222"}, buf)

In [193]:
buf.getvalue()

'{"x": "y"}{"z": "22222"}'

In [ ]:
class TicTac:
    def __init__(self, print_fn=print):
        self._print = print_fn
        
    def move(self):
        self._print("dkwndkwdkwk")
        
        

def test_xxx():
    m_print = mock.Mock()
    tt = TicTac(print_fn=m_print)
    tt.play()
    
    self.assertEqual(
        [mock.call("dkwndkwdkwk")],
        m_print.mock_calls,
    )
    

def test_yyyy():
    tt = TicTac()
    with mock.patch("path_to_TicTac.print") as m_print:
        tt.play()

        self.assertEqual(
            [mock.call("dkwndkwdkwk")],
            m_print.mock_calls,
        )